In [1]:
import numpy as np
import pandas as pd

Side thoughts:
- need to restrict flows to $\mathbf{R}_{++}$

Enumeration of Lakes:\
Lake Superior: 1\
Lake Michigan/Huron : 2\
Lake St. Clair: 3\
Lake Erie: 4\
Lake Ontario: 5\
Sink (including all bodies of water after the St. Lawrence River): 6

Enumeration of Edges:\
St. Mary's River: (1, 2)\
St. Clair River: (2, 3)\
Detroit River: (3, 4)\
Niagra River: (4, 5)\
St. Lawrence River: (5, 6)

We take the following DAG $G = (N, E)$, where $V = \left\{ 1, 2, 3, 4, 5, 6 \right\}$ and $E = \left\{ (1, 2), (2, 3), (3, 4), (4, 5), (5, 6) \right\}$, where $(i, j) \in E$ is an ordered pair representing an edge from node $i$ to node $j$.

In [3]:
edges = [(1, 2), (2, 3), (3, 4), (4, 5), (5, 6)]
N = 6
L = 5
A = np.zeros((N, L))

In [ ]:
edge_counter = 0
for edge in edges:
    i = edge[0] - 1
    j = edge[1] - 1
    A[i, edge_counter] = -1
    A[j, edge_counter] = 1